<a href="https://colab.research.google.com/github/akan72/comp590/blob/master/flask/notebooks/Sentinel2Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import pickle

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/xAHjWTfqYu92spjPsJCbWYhmj_q54OSz9sECjeglxq1ARd2XT_1Q2bY

Successfully saved authorization token.


In [0]:
import folium
import matplotlib.pyplot as plt
from IPython.display import Image

# Folium Utilities 
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [5]:
image = ee.Image('users/danielhirst1998/liveEO/rasterize_on_sentinel_2_labelled_17SLD')
imageCollection = ee.ImageCollection('users/danielhirst1998/liveEO/s2-images')

image = image.select('b1')
image = image.set('class',{ 0: 'urban', 1: 'treerow', 2: 'forest', 3: 'single tree', 4: 'agriculture', 5: 'grassland',6: 'water', 4294967295: 'nodata',});
image = image.updateMask(image.neq(4294967295))

NameError: ignored

In [10]:
# create a palette/legend so we can easily see the labels on the map
palette = [
  'FF0000', # urban // red
  '00FF00', # treerow // green
  '0000FF', # forest //blue
  'FFFF00', # single tree //yellow
  'FF00FF', # agriculture //magenta
  '00FFFF', # grassland // cyan
  'FFFFFF', # water //white
];

model_input = imageCollection.toBands()

#Change names of bands to be more understandable
model_input = model_input.select(
   ['T17SLD_20190922T203454_AOT_10m_b1',
    'T17SLD_20190922T203454_B01_60m_b1',
    'T17SLD_20190922T203454_B02_10m_b1',
    'T17SLD_20190922T203454_B03_10m_b1',
    'T17SLD_20190922T203454_B04_10m_b1',
    'T17SLD_20190922T203454_B05_20m_b1',
    'T17SLD_20190922T203454_B06_20m_b1',
    'T17SLD_20190922T203454_B07_20m_b1',
    'T17SLD_20190922T203454_B08_10m_b1',
    'T17SLD_20190922T203454_B09_60m_b1',
    'T17SLD_20190922T203454_B11_20m_b1',
    'T17SLD_20190922T203454_B12_20m_b1',
    'T17SLD_20190922T203454_B8A_20m_b1',
    'T17SLD_20190922T203454_SCL_20m_b1',
    'T17SLD_20190922T203454_WVP_10m_b1'],
    ['AOT','B01','B02','B03','B04','B05','B06','B07','B08','B09','B11','B12','B8A','SCL','WVP'] 
);

#Combine the unlabelled images with the labels from the labelled image
model_input = model_input.addBands(image.select(["b1"],["label"]))

my_map = folium.Map(
    location=[39.0681, -83.0143]
)

my_map.add_ee_layer(image,
             {'min': 0, 'max': 6, 'palette': palette},
             'classification')

my_map.add_child(folium.LayerControl())

In [0]:
#//---------------------Supervised Classification--------------------------//

# Use these bands for prediction.
# All bands used to have most data possible, doing a combination of bands could help to increase accuracy in the future 
bands =  ['AOT','B01','B02','B03','B04','B05','B06','B07','B08','B09','B11','B12','B8A','SCL','WVP','label'] ;

#//Set scale and numPoint parameters so we can quickly change them for both the training and validation model
scale = 10; #// highest resolution of the s2 bands is 10m, so scale=10 preserves all data for all bands
numPoints = 100000

#Sample the input imagery using a stratified sample
training = model_input.select(bands).stratifiedSample(
    classBand='label',
    scale=scale,
    numPoints=numPoints,
    seed=0,
    region=model_input.geometry());

# Train classifier
classifier = ee.Classifier.cart().train(
    features=training,
    classProperty='label',
    inputProperties=bands);

# .pkl model results
# pickle.dump(classifier, open('models/s2-classifier.pkl','wb'))

In [12]:
#Classify the image with the same bands used for training.
classified = model_input.select(bands).classify(classifier);

# Get a confusion matrix representing resubstitution accuracy.
trainAccuracy = classifier.confusionMatrix()

print('Resubstitution error matrix: '+ str(trainAccuracy.getInfo()))
print('Training Accuracy: ' + str(trainAccuracy.accuracy().getInfo()))

Resubstitution error matrix: [[18616, 8, 24, 0, 188, 16, 236], [1, 4020, 2029, 0, 997, 134, 7], [21, 561, 98183, 0, 935, 294, 6], [11, 34, 15, 7, 82, 17, 0], [30, 65, 87, 0, 93250, 394, 53], [2, 46, 435, 0, 3777, 3302, 4], [316, 5, 5, 0, 52, 10, 45099]]
Training Accuracy: 0.9601388573895103


In [13]:
# Sample the input with a different seed so we can compare the classifier and validate the model.
validation = model_input.select(bands).stratifiedSample(
    classBand='label',
    scale=scale,
    numPoints=numPoints,
    seed=1,
    region=model_input.geometry())

#Classify the validation data.
validated = validation.classify(classifier)

#// Get a confusion matrix representing expected accuracy.
testAccuracy = validated.errorMatrix('label', 'classification')

print('Validation error matrix: ', testAccuracy.getInfo())
print('Validation overall accuracy: ', testAccuracy.accuracy().getInfo())

Validation error matrix:  [[18616, 8, 24, 0, 188, 16, 236], [1, 4020, 2029, 0, 997, 134, 7], [16, 588, 98115, 0, 981, 291, 9], [11, 34, 15, 7, 82, 17, 0], [30, 65, 87, 0, 93250, 394, 53], [2, 46, 435, 0, 3777, 3302, 4], [316, 5, 5, 0, 52, 10, 45099]]
Validation overall accuracy:  0.9598901139098817


In [17]:
my_map = folium.Map(
    location=[39.0681, -83.0143]
)

my_map.add_ee_layer(classified,
                    {'min': 0, 'max': 6, 'palette': palette},
                    'predictions')

my_map.add_children(folium.LayerControl())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  if __name__ == '__main__':
